## 가중치 초기화

### Xavier Normal initialization

$$ W \sim N(0, Var(W)) $$
$$ Var(W) = \sqrt{\frac{2}{n_{in} + n_{out}}} $$

### Xavier Uniform initialization

$$ W \sim U(- \sqrt{\frac{6}{n_{in} + n_{out}}}, + \sqrt{\frac{6}{n_{in} + n_{out}}}) $$

In [10]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [12]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [13]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [14]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [15]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [16]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0215, -0.0894,  0.0598,  ...,  0.0200,  0.0203,  0.1212],
        [ 0.0078,  0.1378,  0.0920,  ...,  0.0975,  0.1458, -0.0302],
        [ 0.1270, -0.1296,  0.1049,  ...,  0.0124,  0.1173, -0.0901],
        ...,
        [ 0.0661, -0.1025,  0.1437,  ...,  0.0784,  0.0977, -0.0396],
        [ 0.0430, -0.1274, -0.0134,  ..., -0.0582,  0.1201,  0.1479],
        [-0.1433,  0.0200, -0.0568,  ...,  0.0787,  0.0428, -0.0036]],
       requires_grad=True)

In [17]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [18]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost.item()
        
    avg_cost /= total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.246791000
Epoch: 0002 cost = 0.092913982
Epoch: 0003 cost = 0.061181259
Epoch: 0004 cost = 0.043839336
Epoch: 0005 cost = 0.032600096
Epoch: 0006 cost = 0.025538676
Epoch: 0007 cost = 0.020199952
Epoch: 0008 cost = 0.019448803
Epoch: 0009 cost = 0.015196425
Epoch: 0010 cost = 0.012831674
Epoch: 0011 cost = 0.014177091
Epoch: 0012 cost = 0.009955725
Epoch: 0013 cost = 0.012739657
Epoch: 0014 cost = 0.011522935
Epoch: 0015 cost = 0.008984332
Learning finished


In [21]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.979699969291687
Label:  5
Prediction:  5


### He Normal initialization

$$ W \sim N(0, Var(W)) $$
$$ Var(W) = \sqrt{\frac{2}{n_{in}}} $$

### He Uniform initialization

$$ W \sim U(- \sqrt{\frac{6}{n_{in}}}, + \sqrt{\frac{6}{n_{in}}}) $$

In [22]:
# He Initialization 
torch.nn.init.kaiming_uniform_(linear1.weight, mode='fan_in', nonlinearity='relu')
torch.nn.init.kaiming_uniform_(linear2.weight, mode='fan_in', nonlinearity='relu')
torch.nn.init.kaiming_uniform_(linear3.weight, mode='fan_in', nonlinearity='relu')

Parameter containing:
tensor([[ 0.0994,  0.1391, -0.0192,  ..., -0.0012,  0.0045,  0.0993],
        [ 0.1422,  0.0913, -0.1280,  ..., -0.0883,  0.1043,  0.0608],
        [-0.0659, -0.1194, -0.0801,  ...,  0.0895, -0.0209, -0.0917],
        ...,
        [ 0.1256,  0.0296,  0.0326,  ..., -0.0182, -0.0230,  0.0749],
        [-0.0263, -0.1397,  0.0329,  ..., -0.0692, -0.0783, -0.1528],
        [ 0.1404,  0.0541, -0.1363,  ..., -0.0020,  0.0851,  0.0032]],
       device='cuda:0', requires_grad=True)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost.item()
        
    avg_cost /= total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.235164701
Epoch: 0002 cost = 0.089944864
Epoch: 0003 cost = 0.057343189
Epoch: 0004 cost = 0.039686907
Epoch: 0005 cost = 0.031052398
Epoch: 0006 cost = 0.025835211
Epoch: 0007 cost = 0.018439866
Epoch: 0008 cost = 0.014824807
Epoch: 0009 cost = 0.015250146
Epoch: 0010 cost = 0.013935009
Epoch: 0011 cost = 0.012784812
Epoch: 0012 cost = 0.012120698
Epoch: 0013 cost = 0.012084293
Epoch: 0014 cost = 0.008108170
Epoch: 0015 cost = 0.010945360
Learning finished


In [25]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9785999655723572
Label:  7
Prediction:  7
